In [1216]:
import pandas as pd
import json
from tqdm.autonotebook import tqdm

In [1217]:
df = pd.read_csv("data/AKG/Test Collection/AKGG/akg_standard_akgg_property_rele.csv")

In [35]:
import json

with open("data/AKG/Formal Run Topics/AKGG_Formal_Run_Topic.json") as json_file:
    data = json.load(json_file)
    qid, query, entity, entityType, action = [], [], [], [], []
    for p in data['queries']:
        qid.append(p['queryId'])
        query.append(p['query'])   
        entity.append(p['entity'])
        entityType.append(' '.join(p['entityTypes']))    
        action.append(p['action'])
topic = pd.DataFrame({"query_id": qid, "query": query, "entity": entity, "entityType": entityType, "action":action})

In [99]:
class AutoVivification(dict):
    """Implementation of perl's autovivification feature."""
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value

In [156]:

with open("data/AKG/Participants Runs/AKGG/akgg-formalrun-cuis.json") as json_file:
    data = json.load(json_file)
    run = AutoVivification()
    for p in data['runs']:
        for res in p['results']:
            for prop in res['properties']:
                run[p['runid']][str(res['queryid'])][str(prop['property'])] = prop['rank']


In [1223]:
df = df.merge(topic, how="inner", on="query_id")
df['query'] = df['query'].str.lower()
df['action'] = df['action'].str.lower()
df['entityType'] = df['entityType'].str.lower()
df['entity'] = df['entity'].str.lower()

In [367]:
import collections
qrel = collections.defaultdict(dict)
for qid, prop, label in df[['query_id', 'property', 'rele_label']].values:
    qrel[str(qid)][str(prop)] = int(label)
    

In [368]:
selected_qids = df[df['query'].isin(list(entity2wiki.keys()))].query_id.unique()

In [449]:
from pyNTCIREVAL import Labeler
from pyNTCIREVAL.metrics import MSnDCG, nERR, nDCG

res = []
for i in qrel:
    
    if int(i) not in selected_qids:
        continue
    # dict of { document ID: relevance level }
    qrels = qrel[str(i)] 
    grades = [1,2,3,4] # a grade for relevance levels 1 and 2 (Note that level 0 is excluded)
#     grades = list(set(qrels.values())) + [0]
    ranked_list = list(run['1'][str(i)].keys()) # a list of document IDs
#     ranked_list = naive[str(i)] # a list of document IDs
#     ranked_list =  {**naive[str(i)],  **run['1'][str(i)]}
#     for k in ranked_list:
#         ranked_list[k] *= -1

    # labeling: [doc_id] -> [(doc_id, rel_level)]
    labeler = Labeler(qrels)
    labeled_ranked_list = labeler.label(ranked_list)

    # compute the number of documents for each relevance level
    rel_level_num = 5
    xrelnum = labeler.compute_per_level_doc_num(rel_level_num)

    # Let's compute nDCG@5
    metric = MSnDCG(xrelnum, grades, cutoff=10)
    result = metric.compute(labeled_ranked_list)
    res.append(result)

print(np.mean(res))

0.5196837307322266


In [307]:
df_wiki = pd.read_csv("data/wikihowSep.csv")
df_wiki['headline'] = df_wiki['headline'].str.replace("\n", "")
df_wiki['title'] = df_wiki['title'].str.replace("How to", "")

In [787]:
df_wiki['overview'] = df_wiki['overview'].astype(str)
df_wiki['headline'] = df_wiki['headline'].astype(str)
df_wiki['text'] = df_wiki['text'].astype(str)
df_wiki['sectionLabel'] = df_wiki['sectionLabel'].astype(str)
df_wiki['title'] = df_wiki['title'].astype(str)

In [1322]:
df_wiki['overview'] = df_wiki['overview'].str.lower()
df_wiki['title'] = df_wiki['title'].str.lower()

In [340]:
%%time
queries = [q.lower() for q in df['query'].unique()]
query2wiki = collections.defaultdict(object)
for entity in tqdm(queries, total=len(queries)):
    df_ = df_wiki[df_wiki.title.str.contains(entity, na=False)]
    if len(df_) > 0:
        query2wiki[entity] = df_


CPU times: user 2min 11s, sys: 668 ms, total: 2min 12s
Wall time: 2min 12s


In [682]:
query2id = dict(zip(df['query'], df['query_id']))
query2entity = dict(zip(df['query'], df['entity']))
query2action = dict(zip(df['query'], df['action']))

In [456]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.
/Users/jarana/anaconda3/envs/keras/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jarana/anaconda3/envs/keras/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/jarana/anaconda3/envs/keras/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/jarana/anacon

In [470]:
all_properties = []
for i in run["1"]:
    all_properties.extend(list(run["1"][i].keys()))

In [1231]:
matched_wiki = df_wiki[df_wiki.text.str.contains('|'.join(queries), na=False)]
wiki_all_text = []
for col in df_wiki.columns:
    wiki_all_text.extend(matched_wiki[col].unique())

In [1242]:
df.entityType.unique()

array(['thing action', 'thing creativework movie',
       'thing creativework musicrecording', 'thing creativework book',
       'thing creativework', 'thing creativework tvseries', 'thing event',
       'thing event sportsevent', 'thing event educationevent',
       'thing intangible', 'thing intangible computerlanguage',
       'thing intangible language', 'thing intangible service',
       'thing intangible service financialproduct',
       'thing medicalentity medicalprocedure',
       'thing medicalentity medicalintangible',
       'thing medicalentity medicalcondition',
       'thing medicalentity anatomicalsystem',
       'thing medicalentity substance drug',
       'thing medicalentity substance',
       'thing medicalentity parasiticdisease',
       'thing organization corporation',
       'thing organization sportsorganization',
       'thing organization broadcastchannel',
       'thing organization governmentorganization',
       'thing organization performinggroup musicgro

In [1245]:
df[['query_id', 'entity', 'entityType', 'action']].drop_duplicates().head()

,query_id,entity,entityType,action
0,1,benchmarking,thing action,have
11,2,bodybuilding,thing action,take
20,3,brainstorming,thing action,brainstorm
31,4,chain smoking,thing action,start
38,5,funding,thing action,get funding


In [1251]:
df[df.entityType == 'thing action']['property'].unique()

array(['startTime', 'result', 'target', 'name', 'actionStatus', 'object',
       'instrument', 'disambiguatingDescription', 'description',
       'participant', 'agent', 'potentialAction', 'endTime',
       'additionalType', 'image', 'location', 'alternateName', 'url',
       'identifier', 'error'], dtype=object)

In [1268]:
df[df.entityType == "thing product"].entity.nunique()

45

In [1264]:
df.groupby("entityType").size().sort_values(ascending=False)

entityType
thing product                                                     822
thing place                                                       434
thing creativework movie                                          379
thing person                                                      330
thing organization corporation                                    256
thing medicalentity medicalcondition                              200
thing event                                                       157
thing organization sportsorganization                             150
thing action                                                      139
thing product vehicle car                                         113
thing medicalentity anatomicalsystem                               82
thing creativework tvseries                                        61
thing intangible                                                   59
thing product vehicle                                              53
thing cre

[' winterize a jet ski',
 ' ski',
 ' clean jet',
 ' marshall a jet',
 ' slalom ski (water ski on one ski)']

In [1330]:
import re

regex = re.compile('[^a-zA-Z0-9]')
#First parameter is the replacement, second parameter is your input string
def proprocessingText(doc):
    doc = regex.sub(' ', doc)
    doc = " ".join([w for w in doc.split() if not w in stop_words])
    return doc

corpus = []
for idx, row in df_wiki[['overview', 'title']].drop_duplicates().iterrows():
    doc = proprocessingText(row['title'] + row['overview'])
    corpus.append(doc)

In [1486]:
def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0) for m in matches]

In [1331]:
from rank_bm25 import BM25Okapi

tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

In [1564]:
qidMS = []
docMS = []
for idx, row in df[['query_id', 'entity', 'action']].drop_duplicates().iterrows():
    query = row['action'] + " " +row['entity']
    tokenized_query = query.split(" ")
    doc = bm25.get_top_n(tokenized_query, corpus, n=1)
    qidMS.append(row['query_id'])
    docMS.append(doc[0])

In [1581]:
docMS2qidMS = {i:j for i, j in zip(docMS, qidMS)}


In [1582]:
docMS2vec = {}
for i in range(0, len(docMS), 20):
    data = getVectors(docMS[i:i+20])
    for i in data:
        docMS2vec[docMS2qidMS[i]] = data[i]

In [1579]:
propMS2vec = {}
unique_props = df.property.unique().tolist()
for i in range(0, len(unique_props), 20):
    data = getVectors(unique_props[i:i+20])
    for i in data:
        propMS2vec[i] = data[i]

In [1606]:
res = []
for idx, row in df[['query_id', 'entity', 'action']].drop_duplicates().iterrows():
    qrels = qrel[str(row['query_id'])]
    
    rank = {}
    for p in unique_props:
        score = cosine_similarity([docMS2vec[row['query_id']]], [propMS2vec[p]])[0][0]
        rank[p] = score
    rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)][:20]

    res.append(evaluate(qrels, rank))
print(np.mean(res))

0.06145364551558405


In [505]:
topic_all_text = []
for col in ["property", "query", "entity", "entityType", "action"]:
    topic_all_text.extend(df[col].unique())

In [1197]:
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
raw_text = [str(i) for i in wiki_all_text + topic_all_text + all_properties]
# corpus = [w for w in " ".join(corpus).split() if not w in stop_words]

In [1173]:
import http.client, urllib.request, urllib.parse, urllib.error, base64
import json

def getVectors(queries):
    headers = {
        # Request headers
        'Content-Type': 'application/json',
        'Ocp-Apim-Subscription-Key': '924c1505854b4da4a6144a1cce92937f',
    }

    params = urllib.parse.urlencode({})
    
    try:
        conn = http.client.HTTPSConnection('api.msturing.org')
#         conn.request("POST", "/gen/encode?%s" % params, '{"queries": ["how to make gingerbread people (in grams)", "test AI"]}', headers)
        conn.request("POST", "/gen/encode?%s" % params, str({"queries": queries}).replace("\'", "\""), headers)
        response = conn.getresponse()
        data = response.read()
        data = json.loads(data)
        conn.close()
    except Exception as e:
        print(data)
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
    
    return {data[i]['query']:data[i]['vector'] for i in range(len(data))}

data = getVectors(list(df.property.unique()))

In [1569]:
da

{'startTime': [-0.8621034,
  0.471932143,
  0.109499335,
  0.3474549,
  0.5611288,
  0.653467238,
  0.5158571,
  0.6874646,
  0.8484774,
  0.39616853,
  -0.258223563,
  -0.4098286,
  0.707813442,
  0.0530262366,
  -0.457165331,
  0.130086452,
  0.3063525,
  0.0211562216,
  0.707310438,
  -0.246828422,
  -0.5732721,
  0.0155811338,
  0.185920238,
  -0.416339785,
  -0.545615554,
  0.9435241,
  -0.322245121,
  -0.674321353,
  0.216301739,
  -0.75832,
  -0.5357005,
  -0.329517245,
  -0.307841927,
  0.172670841,
  -0.114803627,
  0.433733165,
  -0.784344852,
  0.612149954,
  0.292806,
  0.8440723,
  -0.150983617,
  -0.8171532,
  -0.131651759,
  0.480179936,
  -0.169441715,
  0.850239754,
  0.310940683,
  -0.284486234,
  -0.1547503,
  -0.5140779,
  0.305581719,
  -0.7712701,
  -0.7221871,
  0.430559278,
  0.413798571,
  0.31953,
  0.556732655,
  0.339281142,
  0.138164818,
  -0.7151846,
  -0.32026878,
  0.331602156,
  -0.7149628,
  0.10541039,
  -0.724516451,
  -0.390018433,
  -0.5137568,
  

In [1186]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([data["startTime"]], [data["time"]])

KeyError: 'time'

In [1346]:
MAX_NUM_WORDS = 100000
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(corpus)

In [1347]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 101910 unique tokens.


In [1348]:
def get_pretrain_embeddings(path, word_index, EMBEDDING_DIM=300):
    MAX_NUM_WORDS = len(word_index)
    BASE_DIR = path + 'data/'
    GLOVE_DIR = os.path.join(BASE_DIR, 'w2v')
    print('Indexing word vectors.')

    embeddings_index = {}
    with open(os.path.join(GLOVE_DIR, 'glove.42B.300d.txt'), encoding="utf-8") as f:
        for line in f:
            word, coefs = line.split(maxsplit=1)
            coefs = np.fromstring(coefs, 'f', sep=' ')
            embeddings_index[word] = coefs

    print('Found %s word vectors.' % len(embeddings_index))

    print('Preparing embedding matrix.')

    # prepare embedding matrix
    num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
    embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
    found = 0
    for word, i in word_index.items():
        if i > MAX_NUM_WORDS:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if embedding_vector.shape[0] == 0:
                continue
            embedding_matrix[i] = embedding_vector
            found += 1

    print("Token num: %d, Found Tokens: %d" % (len(word_index), found))

    # load pre-trained word embeddings into an Embedding layer
    embedding_layer = Embedding(num_words,
                                EMBEDDING_DIM,
                                embeddings_initializer=Constant(embedding_matrix))

    return embedding_layer

In [1349]:
from Models import getDSSM, getRanker

path = "/Users/jarana/workspace/WikiHow-Task-Based/"
max_words = len(word_index)
embedding_layer = get_pretrain_embeddings(path, word_index)

Indexing word vectors.
Found 1917494 word vectors.
Preparing embedding matrix.
Token num: 101910, Found Tokens: 69394


In [1377]:
from keras.models import Sequential
from keras import Input, Model
from keras.layers import GlobalMaxPooling1D, Dot, LSTM, Dense, Concatenate


def getDSSM(embedding_layer, MAX_SEQUENCE_LENGTH):

    q_inp = Input(shape=(MAX_SEQUENCE_LENGTH,))
    t_inp = Input(shape=(MAX_SEQUENCE_LENGTH,))
#     lstm = LSTM(100)
    q_emb = GlobalMaxPooling1D()(embedding_layer(q_inp))
    t_emb = GlobalMaxPooling1D()(embedding_layer(t_inp))
#     q_emb = lstm(embedding_layer(q_inp))
#     t_emb = lstm(embedding_layer(t_inp))
    pred = Dot(-1, normalize=True)([q_emb, t_emb])
    model = Model([q_inp, t_inp], pred)

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model
MAX_SEQUENCE_LENGTH = int(np.mean([len(i.split()) for i in corpus]))
model = getDSSM(embedding_layer, MAX_SEQUENCE_LENGTH)

In [ ]:
x_task = pad_sequences(tokenizer.texts_to_sequences(data["t"].tolist()), maxlen=MAX_SEQUENCE_LENGTH)
x_query1 = pad_sequences(tokenizer.texts_to_sequences(data["q1"].tolist()), maxlen=MAX_SEQUENCE_LENGTH)
x_query2 = pad_sequences(tokenizer.texts_to_sequences(data["q2"].tolist()), maxlen=MAX_SEQUENCE_LENGTH)
y = np.array([[i,j] for i,j in zip(data.label1.values, data.label2.values)])

In [1357]:
from pyNTCIREVAL import Labeler
from pyNTCIREVAL.metrics import MSnDCG, nERR, nDCG

def evaluate(qrels, ranked_list):
    res = []
    grades = [1,2,3,4] # a grade for relevance levels 1 and 2 (Note that level 0 is excluded)
    labeler = Labeler(qrels)
    labeled_ranked_list = labeler.label(ranked_list)
    rel_level_num = 5
    xrelnum = labeler.compute_per_level_doc_num(rel_level_num)
    metric = MSnDCG(xrelnum, grades, cutoff=10)
    result = metric.compute(labeled_ranked_list)
    return result

In [1421]:
res = []
for idx, row in df[['query_id', 'entity', 'action']].drop_duplicates().iterrows():
    qrels = qrel[str(row['query_id'])]
    cand_properties = list(run['1'][str(row['query_id'])].keys())
    
    
    query = row['action'] + " " +row['entity']
    tokenized_query = query.split(" ")
    doc = bm25.get_top_n(tokenized_query, corpus, n=1)[0]
#     print(qrels)
#     print(cand_properties)
#     cand_properties = list(run['1'][str(query2id[query])].keys())
#     wikis = [str(i) for i in query2wiki[query][col].unique()]
    x_wikis = pad_sequences(tokenizer.texts_to_sequences([doc]), maxlen=MAX_SEQUENCE_LENGTH)
#     print(x_wikis)
    rank = {}
    for prop in cand_properties:
        x_props = pad_sequences(tokenizer.texts_to_sequences([entity+ " " + action + " " + prop]), maxlen=MAX_SEQUENCE_LENGTH)
#         x_props = pad_sequences(tokenizer.texts_to_sequences([prop]), maxlen=MAX_SEQUENCE_LENGTH)
        rank[prop] = model.predict([x_props, x_wikis])[0][0]
    rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)]
    
    res.append(evaluate(qrels, rank))
print(np.mean(res))

0.5165431119083346


In [1423]:
trainIds, testIds = [], []
for name, group in df.groupby("entityType"):
    if group.query_id.nunique() > 1:
        ids = list(group.query_id.unique())
        mid = int(group.query_id.nunique() / 2)
        trainIds.extend(ids[:mid])
        testIds.extend(ids[mid:])
    else:
        ids = list(group.query_id.unique())
        trainIds.extend(ids)

In [1497]:
MAX_PROP_SEQUENCE_LENGTH = max([len(i.split()) for i in props])

In [1498]:
from keras.models import Sequential
from keras import Input, Model
from keras.layers import GlobalMaxPooling1D, Dot, LSTM, Dense, Concatenate


def getPropModel(embedding_layer, PROPERTY_NUN, MAX_QUERY_SEQUENCE_LENGTH, MAX_DOC_SEQUENCE_LENGTH, MAX_PROP_SEQUENCE_LENGTH):

    q_inp = Input(shape=(MAX_QUERY_SEQUENCE_LENGTH,))
    d_inp = Input(shape=(MAX_DOC_SEQUENCE_LENGTH,))
    p_inp = Input(shape=(MAX_PROP_SEQUENCE_LENGTH,))
    
    
    q_lstm = LSTM(100)
    d_lstm = LSTM(100)
#     q_emb = q_lstm(embedding_layer(q_inp))
#     d_emb = d_lstm(embedding_layer(d_inp))
    q_emb = GlobalMaxPooling1D()(embedding_layer(q_inp))
    d_emb = GlobalMaxPooling1D()(embedding_layer(d_inp))
    p_emb = GlobalMaxPooling1D()(embedding_layer(p_inp))
    
    concat = Concatenate()([q_emb, d_emb, p_emb])
    dense = Dense(1, activation='sigmoid')
    pred = dense(concat)
    model = Model([q_inp, d_inp, p_inp], pred)

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [1425]:
from keras.utils import to_categorical

In [1426]:
prop2id = {j:i for i, j in zip(range(df.property.nunique()), df.property.unique())}
id2prop = {i:j for i, j in zip(range(df.property.nunique()), df.property.unique())}

In [1427]:
PROPERTY_NUN = df.property.nunique() 


In [1428]:
MAX_DOC_SEQUENCE_LENGTH = int(np.mean([len(i.split()) for i in corpus]))

In [1560]:
def get_train_data():
    train_id, train_query, train_prop, train_doc, train_y = [], [], [], [], []
    test_id, test_query, test_prop, test_doc, test_y = [], [], [], [], []

    for idx, row in tqdm(df[['query_id', 'entity', 'action']].drop_duplicates().iterrows()):

    #     qrels = qrel[str(row['query_id'])]
    #       convert prop to prop_id
    #     label = to_categorical([prop2id[i] for i in list(qrels.keys())], num_classes=PROPERTY_NUN)
    #     label = np.sum(label, axis=0)
        label = 1

        query = row['action'] + " " +row['entity']
        tokenized_query = query.split(" ")
        doc = bm25.get_top_n(tokenized_query, corpus, n=1)[0]

        x_query = pad_sequences(tokenizer.texts_to_sequences([query]), maxlen=MAX_QUERY_SEQUENCE_LENGTH)[0]
        doc = pad_sequences(tokenizer.texts_to_sequences([doc]), maxlen=MAX_DOC_SEQUENCE_LENGTH)[0]

        pos_props = [" ".join(camel_case_split(i)) for i in df[df.query_id == row['query_id']].property.tolist()]
        for pp in pos_props:
            label = [1,0]
            prop = pad_sequences(tokenizer.texts_to_sequences([pp]), maxlen=MAX_PROP_SEQUENCE_LENGTH)[0]

            train_prop.append(prop) if row['query_id'] in trainIds else test_prop.append(prop)

            neg_prop = random.sample(props, k=1)
            while neg_prop in pos_props:
                neg_prop = random.sample(props, k=1)
            neg_prop = pad_sequences(tokenizer.texts_to_sequences([neg_prop]), maxlen=MAX_PROP_SEQUENCE_LENGTH)[0]
            train_prop.append(neg_prop) if row['query_id'] in trainIds else test_prop.append(neg_prop)

            for i in range(2):
                train_query.append(x_query) if row['query_id'] in trainIds else test_query.append(x_query)
                train_doc.append(doc) if row['query_id'] in trainIds else test_doc.append(doc)
                train_id.append(row['query_id']) if row['query_id'] in trainIds else test_id.append(row['query_id'])

            train_y.extend(label) if row['query_id'] in trainIds else test_y.extend(label)
    #         rank = {}
    #         for prop in cand_properties:
    #             x_props = pad_sequences(tokenizer.texts_to_sequences([entity+ " " + action + " " + prop] * len(x_wikis)), maxlen=MAX_SEQUENCE_LENGTH)
    #             rank[prop] = np.max(model.predict([x_props, x_wikis]))
    #         rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)]
    #         res.append(evaluate(qrels, rank))
    train_query = np.array(train_query)
    train_doc = np.array(train_doc)
    train_prop = np.array(train_prop)
    train_y = np.array(train_y)
    test_query = np.array(test_query)
    test_doc = np.array(test_doc)
    test_prop = np.array(test_prop)
    test_y = np.array(test_y)
    
    return train_query, train_doc, train_prop, train_y, train_id, test_query, test_doc, test_prop, test_y, test_id


In [1552]:
model = getPropModel(embedding_layer, PROPERTY_NUN, MAX_QUERY_SEQUENCE_LENGTH, MAX_DOC_SEQUENCE_LENGTH, MAX_PROP_SEQUENCE_LENGTH)


In [1561]:
def eval_model(model):
    pred = model.predict([test_query, test_doc])  
    sort_pred = np.argsort(pred)[:,::-1][:,:20]
    runLSTM = {}
    for i, j in zip(test_id, sort_pred):
        runLSTM[i] = [id2prop[k] for k in j]
    for col in df_wiki.columns:
        res = []
        for qid in testIds:
            qrels = qrel[str(qid)]
            cand_properties = runLSTM[qid]
            res.append(evaluate(qrels, cand_properties))
        print(col, np.mean(res))    
        break
    

for i in range(5):
    train_query, train_doc, train_prop, train_y, train_id, test_query, test_doc, test_prop, test_y, test_id = get_train_data()
    model.fit([train_query, train_doc, train_prop], train_y, validation_data=([test_query, test_doc, test_prop], test_y), verbose=2)
#     eval_model(model)


Train on 3878 samples, validate on 3714 samples
Epoch 1/1
 - 41s - loss: 0.4636 - acc: 0.7035 - val_loss: 0.4486 - val_acc: 0.7019



Train on 3878 samples, validate on 3714 samples
Epoch 1/1
 - 40s - loss: 0.4407 - acc: 0.7112 - val_loss: 0.4351 - val_acc: 0.7146



Train on 3878 samples, validate on 3714 samples
Epoch 1/1
 - 37s - loss: 0.4361 - acc: 0.7060 - val_loss: 0.4286 - val_acc: 0.7132



Train on 3878 samples, validate on 3714 samples
Epoch 1/1
 - 40s - loss: 0.4239 - acc: 0.7215 - val_loss: 0.4198 - val_acc: 0.7251



Train on 3878 samples, validate on 3714 samples
Epoch 1/1
 - 2302s - loss: 0.4239 - acc: 0.7305 - val_loss: 0.4232 - val_acc: 0.7186


In [1465]:
for qid in testIds:
    qrels = qrel[str(qid)]
    cand_properties = list(run['1'][str(qid)].keys())
    res.append(evaluate(qrels, cand_properties))
print(np.mean(res))    
    



0.529892936164067
